<a href="https://colab.research.google.com/github/sur-sakthy/computer-vision/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp '/content/drive/MyDrive/EE544 Computer Vision/imagewoof2-320.tgz' ./

In [2]:
# remove %%capture command to show output from unzipping
%%capture
!tar zxvf /content/imagewoof2-320.tgz

In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, AUC, Recall, Accuracy
from tensorflow.keras.optimizers import Adam

In [4]:
PATH_TO_TRAIN_DATA = '/content/imagewoof2-320/train'
PATH_TO_VAL_DATA = '/content/imagewoof2-320/val'
IMG_WIDTH=224
IMG_HEIGHT=224

In [5]:
all_classes = os.listdir(PATH_TO_TRAIN_DATA)
all_classes

['n02105641',
 'n02087394',
 'n02096294',
 'n02115641',
 'n02088364',
 'n02086240',
 'n02099601',
 'n02089973',
 'n02093754',
 'n02111889']

In [6]:
all_classes = os.listdir(PATH_TO_VAL_DATA)
all_classes

['n02105641',
 'n02087394',
 'n02096294',
 'n02115641',
 'n02088364',
 'n02086240',
 'n02099601',
 'n02089973',
 'n02093754',
 'n02111889']

In [7]:
img_data_array = []
class_name = []

In [8]:
def load_dataset(img_folder): 
    for dir in os.listdir(img_folder):
      if dir == 'n02105641' or dir == 'n02087394' or dir == 'n02096294':
        for file in os.listdir(os.path.join(img_folder, dir)):
       
            image_path = os.path.join(img_folder, dir, file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir)

In [9]:
load_dataset(PATH_TO_TRAIN_DATA)
load_dataset(PATH_TO_VAL_DATA)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(img_data_array, class_name, test_size=0.30, random_state=42)

In [11]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

In [12]:
target_dict = { k: v for v, k in enumerate(np.unique(y_train)) }
target_dict

{'n02087394': 0, 'n02096294': 1, 'n02105641': 2}

In [13]:
def get_target_val(target_dict, target_data):
  return [ target_dict[target_data[i]] for i in range(len(target_data)) ]

In [14]:
def build_model():
  base_model = ResNet50(include_top=False, weights='imagenet')

  for layer in base_model.layers[:-33]: 
    layer.trainable = False

  # add a global spatial average pooling layer
  x = GlobalAveragePooling2D()(base_model.output)

  # add fully-connected
  x = Dense(512, activation='relu', name='fc-1')(x) 
  x = Dense(256, activation='relu', name='fc-2')(x)

  # a softmax layer for 3 classes
  out = Dense(1, activation='softmax', name='output_layer')(x)

  model = Model(base_model.input, out)

  model.compile(
      optimizer=Adam(),
      loss='categorical_crossentropy',
      metrics=[
          Accuracy(),
          Precision(), 
          Recall(),
          AUC()
        ]
  )

  return model

In [15]:
custom_resnet_model = build_model()

In [16]:
custom_resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [17]:
# remove command to show output from unzipping
%%capture 
for i in range(len(custom_resnet_model.layers)): 
  print(str(i) + ': ' + str(custom_resnet_model.layers[i].trainable))

In [18]:
history = custom_resnet_model.fit(
    x=tf.cast(np.array(X_train), tf.float64), 
    y=tf.cast(list(map(int, get_target_val(target_dict, y_train))), tf.int32), 
    epochs=5,
    validation_data=(
        tf.cast(np.array(X_val), tf.float64), 
        tf.cast(list(map(int, get_target_val(target_dict, y_val))), tf.int32)
    )
)

Epoch 1/5
89/89 [==============================] - 976s 11s/step - loss: 0.0000e+00 - accuracy: 0.0011 - precision: 0.6674 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.6633 - val_recall: 1.0000 - val_auc: 0.5000
Epoch 2/5
89/89 [==============================] - 918s 10s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - precision: 0.6674 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.6633 - val_recall: 1.0000 - val_auc: 0.5000
Epoch 3/5
89/89 [==============================] - 911s 10s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - precision: 0.6674 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision: 0.6633 - val_recall: 1.0000 - val_auc: 0.5000
Epoch 4/5
89/89 [==============================] - 906s 10s/step - loss: 0.0000e+00 - accuracy: 3.5273e-04 - precision: 0.6674 - recall: 1.0000 - auc: 0.5000 - val_loss: 0.0000e+00 - val_accu

In [19]:
custom_resnet_model.save_weights('/content/drive/MyDrive/EE544 Computer Vision/task #2.h5')